In [1]:
import kts
kts.config.storage_path = "/mnt/tenTB/avalur/santander_storage/"

In [2]:
from kts.validation.validator import *
from kts.validation.split import *
from kts.zoo.binary_classification import *
from kts.feature import stl
from kts.feature.decorators import *
from kts.optimization.optimizer import *
from kts.feature.storage import feature_list as features
from kts.feature.storage import FeatureSet
from kts.validation.experiment import experiment_list as experiments
from kts.storage.cache_utils import clear_storage

In [3]:
import numpy as np
import pandas as pd
import feather
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm as tm
from tqdm import trange

In [4]:
%%time
train = feather.read_dataframe('../input/train.fth')
test = feather.read_dataframe('../input/test.fth')
oof = train[['ID_code', 'target']]
oof['predict'] = 0
predictions = test[['ID_code']]
val_aucs = []
feature_importance_df = pd.DataFrame()
train.set_index('ID_code', inplace=True)
test.set_index('ID_code', inplace=True)

CPU times: user 251 ms, sys: 742 ms, total: 993 ms
Wall time: 3.15 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
test.shape

(200000, 200)

In [6]:
test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [ ]:
deregister('mean_encoding')
# set(np.linspace(0, 1, 11))
# list(range(1, 20))

In [28]:
@register
def mean_encoding(df):    
    def make_bins(col, n_bins=10):
        borders = set(np.linspace(col.min(), col.max(), n_bins+1))
        for p in np.linspace(0, 1, n_bins+1):
            borders.add(col.quantile(p))
        labels = range(1, len(borders))
        return sorted(list(borders)), list(labels)
    cols = stl.empty_like(df)
    for col in df:
        if col != 'target':
            bins, labels = make_bins(df[col])
            cols[f'binned_{col}'] = pd.cut(df[col], bins=bins, labels=labels)
    col_names = cols.columns.values.tolist()
    cols['target'] = df['target']
    return stl.target_encoding(col_names, 'target')(cols)

In [33]:
fs_ue = FeatureSet(
    [stl.column_dropper(['target']), unique_encoding], 
    df_input=train, 
    target_column='target'
)

In [34]:
# Initialize some models from kts.zoo
# You can set any parameters for a model by passing a dictionary 
xgb = XGBClassifier() # default XGBoost
lgb = LGBMClassifier()
rf = RandomForestClassifier()
lr = LogisticRegression()

In [35]:
from sklearn.metrics import roc_auc_score
target = train.target.values

# Validation is easy with kts
# You just need a Splitter and a Validator

skf = SKF(target, n_folds=5, seed=7) # Stratified k-fold
val_skf = Validator(skf, roc_auc_score) # Validator needs a splitter and a metric

In [36]:
experiments

Experiments: [
(1.0, std:0.0: 
	Model: ens_lgb_c3_x5)
(0.86459, std:0.002: 
	Model: ens_lgb_c3_x5)
(0.85918, std:0.001: 
	Model: ens_lr_93_x5)
(0.85298, std:0.002: 
	Model: ens_lgb_97_x5)
(0.84293, std:0.002: 
	Model: ens_lgb_97_x5)
(0.83169, std:0.002: 
	Model: ens_xgb_76_x5)
(0.69933, std:0.004: 
	Model: ens_rf_c9_x5)
]

In [37]:
lgb_fast = LGBMClassifier(n_jobs=2)
val_skf.score(lgb_fast, fs_ue)

Val of lgb_c3:   0%|          | 0/5 [00:00<?, ?it/s]

[1]	valid_0's binary_logloss: 0.32221
[2]	valid_0's binary_logloss: 0.318998
[3]	valid_0's binary_logloss: 0.316175
[4]	valid_0's binary_logloss: 0.313697
[5]	valid_0's binary_logloss: 0.311337
[6]	valid_0's binary_logloss: 0.309015
[7]	valid_0's binary_logloss: 0.306671
[8]	valid_0's binary_logloss: 0.304628
[9]	valid_0's binary_logloss: 0.302811
[10]	valid_0's binary_logloss: 0.30072
[11]	valid_0's binary_logloss: 0.298825
[12]	valid_0's binary_logloss: 0.297184
[13]	valid_0's binary_logloss: 0.295625
[14]	valid_0's binary_logloss: 0.294362
[15]	valid_0's binary_logloss: 0.292739
[16]	valid_0's binary_logloss: 0.291452
[17]	valid_0's binary_logloss: 0.290116
[18]	valid_0's binary_logloss: 0.288793
[19]	valid_0's binary_logloss: 0.287606
[20]	valid_0's binary_logloss: 0.286374
[21]	valid_0's binary_logloss: 0.285231
[22]	valid_0's binary_logloss: 0.284259
[23]	valid_0's binary_logloss: 0.283076
[24]	valid_0's binary_logloss: 0.28213
[25]	valid_0's binary_logloss: 0.281135
[26]	valid_0

Val of lgb_c3:  20%|██        | 1/5 [00:29<01:57, 29.44s/it, score: 0.8731135324185056]

[1]	valid_0's binary_logloss: 0.32201
[2]	valid_0's binary_logloss: 0.318688
[3]	valid_0's binary_logloss: 0.31603
[4]	valid_0's binary_logloss: 0.313449
[5]	valid_0's binary_logloss: 0.311026
[6]	valid_0's binary_logloss: 0.30883
[7]	valid_0's binary_logloss: 0.30654
[8]	valid_0's binary_logloss: 0.304726
[9]	valid_0's binary_logloss: 0.302921
[10]	valid_0's binary_logloss: 0.30114
[11]	valid_0's binary_logloss: 0.299164
[12]	valid_0's binary_logloss: 0.297525
[13]	valid_0's binary_logloss: 0.296012
[14]	valid_0's binary_logloss: 0.294443
[15]	valid_0's binary_logloss: 0.293201
[16]	valid_0's binary_logloss: 0.291692
[17]	valid_0's binary_logloss: 0.290242
[18]	valid_0's binary_logloss: 0.288819
[19]	valid_0's binary_logloss: 0.287398
[20]	valid_0's binary_logloss: 0.28624
[21]	valid_0's binary_logloss: 0.285241
[22]	valid_0's binary_logloss: 0.284005
[23]	valid_0's binary_logloss: 0.282867
[24]	valid_0's binary_logloss: 0.281884
[25]	valid_0's binary_logloss: 0.280789
[26]	valid_0's 

Val of lgb_c3:  40%|████      | 2/5 [00:57<01:27, 29.04s/it, score: 0.87112828412342]

...

[1]	valid_0's binary_logloss: 0.322072
[2]	valid_0's binary_logloss: 0.318501
[3]	valid_0's binary_logloss: 0.315842
[4]	valid_0's binary_logloss: 0.313161
[5]	valid_0's binary_logloss: 0.310834
[6]	valid_0's binary_logloss: 0.30851
[7]	valid_0's binary_logloss: 0.306263
[8]	valid_0's binary_logloss: 0.304177
[9]	valid_0's binary_logloss: 0.302191
[10]	valid_0's binary_logloss: 0.300079
[11]	valid_0's binary_logloss: 0.298063
[12]	valid_0's binary_logloss: 0.296531
[13]	valid_0's binary_logloss: 0.294995
[14]	valid_0's binary_logloss: 0.293673
[15]	valid_0's binary_logloss: 0.292062
[16]	valid_0's binary_logloss: 0.290514
[17]	valid_0's binary_logloss: 0.289251
[18]	valid_0's binary_logloss: 0.287797
[19]	valid_0's binary_logloss: 0.286553
[20]	valid_0's binary_logloss: 0.285364
[21]	valid_0's binary_logloss: 0.284203
[22]	valid_0's binary_logloss: 0.282981
[23]	valid_0's binary_logloss: 0.281847
[24]	valid_0's binary_logloss: 0.280694
[25]	valid_0's binary_logloss: 0.27955
[26]	valid_

Val of lgb_c3:  80%|████████  | 4/5 [01:53<00:28, 28.48s/it, score: 0.8715292021551045]

[1]	valid_0's binary_logloss: 0.322195
[2]	valid_0's binary_logloss: 0.31886
[3]	valid_0's binary_logloss: 0.31636
[4]	valid_0's binary_logloss: 0.313678
[5]	valid_0's binary_logloss: 0.311535
[6]	valid_0's binary_logloss: 0.309183
[7]	valid_0's binary_logloss: 0.307055
[8]	valid_0's binary_logloss: 0.304928
[9]	valid_0's binary_logloss: 0.303009
[10]	valid_0's binary_logloss: 0.301409
[11]	valid_0's binary_logloss: 0.299871
[12]	valid_0's binary_logloss: 0.298066
[13]	valid_0's binary_logloss: 0.296445
[14]	valid_0's binary_logloss: 0.29475
[15]	valid_0's binary_logloss: 0.293266
[16]	valid_0's binary_logloss: 0.291897
[17]	valid_0's binary_logloss: 0.290587
[18]	valid_0's binary_logloss: 0.289231
[19]	valid_0's binary_logloss: 0.288088
[20]	valid_0's binary_logloss: 0.286964
[21]	valid_0's binary_logloss: 0.285564
[22]	valid_0's binary_logloss: 0.284309
[23]	valid_0's binary_logloss: 0.283286
[24]	valid_0's binary_logloss: 0.282147
[25]	valid_0's binary_logloss: 0.281057
[26]	valid_0

Val of lgb_c3: 100%|██████████| 5/5 [02:29<00:00, 30.80s/it, score: 0.8710430268196048]


0.8710430268196048

In [38]:
experiments

Experiments: [
(1.0, std:0.0: 
	Model: ens_lgb_c3_x5)
(0.87104, std:0.002: 
	Model: ens_lgb_c3_x5)
(0.86459, std:0.002: 
	Model: ens_lgb_c3_x5)
(0.85918, std:0.001: 
	Model: ens_lr_93_x5)
(0.85298, std:0.002: 
	Model: ens_lgb_97_x5)
(0.84293, std:0.002: 
	Model: ens_lgb_97_x5)
(0.83169, std:0.002: 
	Model: ens_xgb_76_x5)
(0.69933, std:0.004: 
	Model: ens_rf_c9_x5)
]